# DS-SF-34 | 03 | Databases, Scrapping, and APIs | Codealong | Starter Code

# Part A | RDBMS Databases and `SQLite`

In [50]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-03-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [4]:
pd.io.sql.read_sql(
'''
SELECT [id], [address]
    FROM properties
    LIMIT 10
;
''', con = db)

,id,address
0,15063471,"55 Vandewater St APT 9, San Francisco, CA"
1,15063505,"740 Francisco St, San Francisco, CA"
2,15063609,"819 Francisco St, San Francisco, CA"
3,15064044,"199 Chestnut St APT 5, San Francisco, CA"
4,15064257,"111 Chestnut St APT 403, San Francisco, CA"
5,15064295,"111 Chestnut St APT 702, San Francisco, CA"
6,15064391,"1821 Grant Ave APT 101, San Francisco, CA"
7,15064536,"2300 Leavenworth St, San Francisco, CA"
8,15064640,"1047-1049 Lombard St, San Francisco, CA"
9,15064669,"1055 Lombard St # C, San Francisco, CA"


### `SELECT`

In [6]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM transactions
;
''', con = db)

,id,date_of_sale,sale_price,sale_price_unit
0,15063471,2015-12-04 00:00:00,710000.00,$
1,15063505,2015-11-30 00:00:00,2.15,$M
2,15063609,2015-11-12 00:00:00,5.60,$M
3,15064044,2015-12-11 00:00:00,1.50,$M
4,15064257,2016-01-15 00:00:00,970000.00,$
...,...,...,...,...
995,2124214951,2016-01-15 00:00:00,390000.00,$
996,2126960082,2015-11-20 00:00:00,860000.00,$
997,2128308939,2015-12-10 00:00:00,830000.00,$
998,2131957929,2015-12-15 00:00:00,835000.00,$


### `WHERE`

In [9]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15063505,2.15
1,15063609,5.60
2,15064044,1.50
3,15064536,2.83
4,15064640,4.05
...,...,...
570,2106229715,1.24
571,2108411930,1.73
572,2108546010,1.40
573,2121221385,1.43


> #### Here's let's convert the unit of `sale_price` from \$ to \$M

In [12]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
--    LIMIT 2

UNION ALL

SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
995,2106229715,1.240
996,2108411930,1.730
997,2108546010,1.400
998,2121221385,1.430


### `UNION`

> #### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [13]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
995,2106229715,1.240
996,2108411930,1.730
997,2108546010,1.400
998,2121221385,1.430


### `WITH`

In [14]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,id,sale_price
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
995,2106229715,1.240
996,2108411930,1.730
997,2108546010,1.400
998,2121221385,1.430


### `JOIN`

In [15]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

,beds,sale_price
0,1.0,0.710
1,2.0,0.970
2,2.0,0.940
3,1.0,0.835
4,1.0,0.800
...,...,...
995,3.0,1.240
996,2.0,1.730
997,2.0,1.400
998,NaN,1.430


### `WITH` (take 2)

In [16]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM beds_normalized_sale_prices
;
''', con = db)

,beds,sale_price
0,1.0,0.710
1,2.0,0.970
2,2.0,0.940
3,1.0,0.835
4,1.0,0.800
...,...,...
995,3.0,1.240
996,2.0,1.730
997,2.0,1.400
998,NaN,1.430


In [18]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

--use from table_1, table_2 instead of a join

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties, normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM beds_normalized_sale_prices
;
''', con = db)

,beds,sale_price
0,1.0,0.710
1,2.0,0.970
2,2.0,0.940
3,1.0,0.835
4,1.0,0.800
...,...,...
995,3.0,1.240
996,2.0,1.730
997,2.0,1.400
998,NaN,1.430


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bedrooms

In [22]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT AVG(sale_price) AS mean_sale_price
    FROM beds_normalized_sale_prices
    GROUP BY beds
;
''', con = db)

,mean_sale_price
0,1.494858
1,0.820425
2,1.192398
3,1.404323
4,1.879330
5,2.867720
6,2.587222
7,5.543167
8,1.960000
9,1.766667


Note: `beds = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [37]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.is_a_studio AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT AVG(sale_price)
    FROM beds_normalized_slae_prices
    WHERE is_a_studio = 1
;
''', con = db)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



DatabaseError: Execution failed on sql '
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.is_a_studio AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT AVG(sale_price)
    FROM beds_normalized_slae_prices
    WHERE is_a_studio = 1
;
': no such table: beds_normalized_slae_prices

> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [41]:
pd.io.sql.read_sql(
'''
WITH same_lat_long AS
  (select latitude, longitude, count(*) as num_with_latlong
  from properties
  group by latitude, longitude
  having num_with_latlong > 1
  ),
 
  lat_long_groups AS
  (select a.latitude, a.longitude, address, id
  from properties a
  join same_lat_long b
  on a.latitude=b.latitude and a.longitude=b.longitude
  )
 
select a.latitude, a.longitude, a.address as address1, b.address as address2
from lat_long_groups a
  join lat_long_groups b
  on a.latitude=b.latitude and a.longitude=b.longitude
  where a.id <> b.id  
order by a.latitude, a.longitude, a.address
;
''', con=db)

,a.latitude,a.longitude,address1,address2
0,37710480,-122467645,"3981 Alemany Blvd APT 206, San Francisco, CA","8300 Oceanview Ter APT 306, San Francisco, CA"
1,37710480,-122467645,"3981 Alemany Blvd APT 206, San Francisco, CA","8400 Oceanview Ter APT 218, San Francisco, CA"
2,37710480,-122467645,"8300 Oceanview Ter APT 306, San Francisco, CA","3981 Alemany Blvd APT 206, San Francisco, CA"
3,37710480,-122467645,"8300 Oceanview Ter APT 306, San Francisco, CA","8400 Oceanview Ter APT 218, San Francisco, CA"
4,37710480,-122467645,"8400 Oceanview Ter APT 218, San Francisco, CA","3981 Alemany Blvd APT 206, San Francisco, CA"
...,...,...,...,...
211,37800573,-122424849,"2701 Van Ness Ave APT 702, San Francisco, CA","2701 Van Ness Ave APT 207, San Francisco, CA"
212,37804240,-122405509,"111 Chestnut St APT 403, San Francisco, CA","111 Chestnut St APT 702, San Francisco, CA"
213,37804240,-122405509,"111 Chestnut St APT 702, San Francisco, CA","111 Chestnut St APT 403, San Francisco, CA"
214,37804334,-122412450,"445 Francisco St APT 304, San Francisco, CA","530 Chestnut St APT 407, San Francisco, CA"


In [49]:
pd.io.sql.read_sql(
'''
WITH id_lat_long AS (SELECT id, latitude, longitude
    FROM properties)

SELECT a.id AS a_id, b.id AS b_id
    FROM id_lat_long AS a, id_lat_long AS b
    WHERE (a.latitude = b.latitude) AND (a.longitude = b.longitude)
    AND (a_id != b_id)
    AND (a.id < b.id)
;
''', con=db)

,a_id,b_id
0,15064257,15064295
1,15066001,15066036
2,15067583,15067604
3,15067890,15067896
4,15068514,15068551
...,...,...
103,124397844,124397860
104,124850278,124862135
105,125156575,2100850402
106,2100643449,2100786271
